In [272]:
import planeclassifier as pc
import numpy as np
import os

In [273]:
path = os.getcwd()
# encode Bomber:0, Fighter:1
data = np.loadtxt(path+'/plane_class_data.csv',
                  skiprows=1,
                  delimiter=",",
                  usecols = [0, 1, 2],
                  converters = {2: lambda p: 0 if p == b'Bomber' else 1}
                 )

In [274]:
data

array([[ 1. ,  0.1,  0. ],
       [ 2. ,  0.2,  0. ],
       [ 0.1,  0.3,  1. ],
       [ 2. ,  0.3,  0. ],
       [ 0.2,  0.4,  1. ],
       [ 3. ,  0.4,  0. ],
       [ 0.1,  0.5,  1. ],
       [ 1.5,  0.5,  0. ],
       [ 0.5,  0.6,  1. ],
       [ 1.6,  0.7,  1. ]])

### Trying with half training and testing

In [275]:
training_data = data[:5,:2]
training_target = data[:5, 2:3]
test_data = data[5:, :2]
test_target = data[5:, 2:3]

In [276]:
#train
perceptron = pc.SequentialPerceptron(training_data, training_target)

# predict
perceptron.train_perceptron(training_data, training_target, 0.25, 3)

In [277]:
# print matrix
print(perceptron.confusion_matrix(test_data, test_target))

[[ 2.  3.]
 [ 0.  0.]]


From the Confusion Matrix The system predicts correctly for the Bomber Planes

In [278]:
print(test_data)

[[ 3.   0.4]
 [ 0.1  0.5]
 [ 1.5  0.5]
 [ 0.5  0.6]
 [ 1.6  0.7]]


In [279]:
print(test_target)

[[ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]]


### Next, let's try to increase the size of the training data set, so we have 8 training sets

In [280]:
training_data = data[0:8, :2]
training_target = data[0:8, 2:3]
test_data = data[8:10, :2]
test_target = data[8:10, 2:3]

In [281]:
print(test_target)

[[ 1.]
 [ 1.]]


### From the test data above, we should expect all the planes to be Fighters

In [282]:
perceptron = pc.SequentialPerceptron(training_data, training_target)

# predict
perceptron.train_perceptron(training_data, training_target, 0.25, 3)

In [283]:
# print matrix
print (perceptron.confusion_matrix(test_data, test_target))

[[ 0.  1.]
 [ 0.  1.]]


### The algorithm wrongly predicts the type of the Fighter plane

##### Since we only have two targets to predict, we can make the algorithm learn more of one type so it correctly predicts by negating the other element.

In [284]:
training_data = data[0:9, :2]
training_target = data[0:9, 2:3]
test_data = data[9:10, :2]
test_target = data[9:10, 2:3]

In [285]:
perceptron = pc.SequentialPerceptron(training_data, training_target)

# predict
perceptron.train_perceptron(training_data, training_target, 0.25, 3)

In [286]:
# print matrix
print (perceptron.confusion_matrix(test_data, test_target))

[[ 0.  1.]
 [ 0.  0.]]


#### Still the problem is not yet solved
For this case we use more of the Bomber data in the training and see how it performs, and we will do this manually for now

In [287]:
training_data = data[[0, 1, 2,  4, 8, 9, 6, 3],:2]
training_target = data[[0, 1, 2,  4, 8, 9, 6, 3], 2:3]
test_data = data[[5, 7], :2]
test_target = data[[5, 7], 2:3]

In [288]:
perceptron = pc.SequentialPerceptron(training_data, training_target)

# predict
perceptron.train_perceptron(training_data, training_target, 0.25, 3)

In [289]:
# print matrix
print (perceptron.confusion_matrix(test_data, test_target))

[[ 1.  0.]
 [ 1.  0.]]


#### Improvements:
* Normalization:
    - We found no need to normalize the data because the values were small
* Get more training data so the system learns different combinations of features and be able to predict correctly.
* Let the algorithm Learn more of just one type so the it predicts correctly on one type and so automatically everything else is not taken to be of the other type

### What if we duplicate the data and do some transformations to the duplicate data so it looks different and try to predict?

In [290]:
data = np.concatenate((data, data), axis=0)
data.shape

(20, 3)

### Reshuffle the data

In [291]:
idx = np.arange(data.shape[0])
np.random.shuffle(idx)
data = data[idx, :]

In [220]:
#### Transforming the data randomly

In [292]:
transformer = np.random.rand(20, 2)* 3/2-0.76
dataT = data[:, :2] * transformer
dataT.shape
targets = np.reshape(data[:, 2], (20,1))

In [293]:
data2 = np.concatenate((dataT, targets), axis=1)
data2.shape

(20, 3)

In [294]:
training = data2[0:15, :2]
training_target = np.reshape(data2[0:15, 2], (15,1))
testing = data2[15:20, :2]
testing_target = data2[15:20, :2]

In [295]:
perceptron = pc.SequentialPerceptron(training, training_target)

# predict
perceptron.train_perceptron(training, training_target, 0.003, 10)

In [296]:
# print matrix
print (perceptron.confusion_matrix(testing, testing_target))

[[ 2.  3.]
 [ 0.  0.]]


## Early-stopping maybe?

In [297]:
training = data2[0:10, :2]
training_target = np.reshape(data2[0:10, 2], (10,1))
valid = data2[10:17, :2]
validt = data2[10:17, 2]
testing = data2[17:20, :2]
testing_target = data2[17:20, :2]

In [298]:
import mlp_algorithm as backprop

In [299]:
net = backprop.mlp(inputs=training,targets=training_target,nhidden=5,outtype='linear')
net.earlystopping(training,training_target,valid,validt,0.1)
net.confmat(testing,testing_target)

1
Iteration:  0  Error:  2.27535106955
2
Iteration:  0  Error:  1.15331237346
3
Iteration:  0  Error:  1.057066532
4
Iteration:  0  Error:  0.686822201558
Stopped 9.16358426916 6.35812618711 6.26967181117
Confusion matrix is:
[[ 2.  1.]
 [ 0.  0.]]
Percentage Correct:  66.6666666667
